In [ ]:
# -*- coding: utf-8 -*-
"""web_heart_data.ipynb"""

import streamlit as st
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import shap

# Configuration de la page
st.set_page_config(page_title="Prédiction de mortalité cardiaque", page_icon="❤️", layout="wide")
st.title("PRÉDICTION DE MORTALITÉ CARDIAQUE PAR MACHINE LEARNING")

# Fonction pour charger et préparer les données
@st.cache_data
def load_data():
    # Télécharger les données si nécessaire
    try:
        df = pd.read_csv('heart_failure_clinical_records_dataset excel.csv')
    except:
        import urllib.request
        url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00519/heart_failure_clinical_records_dataset.csv"
        urllib.request.urlretrieve(url, "heart_failure_clinical_records_dataset.csv")
        df = pd.read_csv('heart_failure_clinical_records_dataset excel.csv')
    return df

# Fonction pour entraîner le modèle
@st.cache_resource
def train_model(df):
    X = df.drop('DEATH_EVENT', axis=1)
    y = df['DEATH_EVENT']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    scaler = StandardScaler().fit(X_train)
    X_train_scaled = scaler.transform(X_train)

    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train_scaled, y_train)

    explainer = shap.TreeExplainer(model)

    return model, scaler, explainer

# Chargement des données
with st.spinner("Chargement des données..."):
    df = load_data()

# Entraînement du modèle
with st.spinner("Entraînement du modèle..."):
    model, scaler, explainer = train_model(df)

# Afficher les données
with st.expander("Voir les données"):
    st.dataframe(df)

# Interface pour les entrées utilisateur
st.header("Données du patient")
st.markdown("Entrez les informations cliniques du patient pour évaluer son risque")

col1, col2 = st.columns(2)

with col1:
    name = st.text_input("Nom du patient", "")
    age = st.slider("Âge", 20, 100, 65)
    gender = st.radio("Genre", ["Femme", "Homme"])
    anaemia = st.checkbox("Anémie")
    diabetes = st.checkbox("Diabète")
    high_blood_pressure = st.checkbox("Hypertension")

with col2:
    ejection_fraction = st.slider("Fraction d'éjection (%)", 10, 80, 35)
    creatinine = st.slider("Créatinine sérique (mg/dL)", 0.5, 10.0, 1.2, 0.1)
    sodium = st.slider("Sodium sérique (mEq/L)", 110, 150, 135)
    platelets = st.slider("Plaquettes (kiloplaquettes/mL)", 25000, 850000, 250000, 10000) / 1000
    creatinine_phosphokinase = st.slider("CPK (U/L)", 20, 8000, 150)
    smoking = st.checkbox("Fumeur")
    time = st.slider("Période de suivi (jours)", 1, 300, 100)

# Bouton de prédiction
predict_button = st.button("Calculer le risque de mortalité")

if predict_button:
    # Préparation des données d'entrée
    input_data = {
        'age': age,
        'anaemia': 1 if anaemia else 0,
        'creatinine_phosphokinase': creatinine_phosphokinase,
        'diabetes': 1 if diabetes else 0,
        'ejection_fraction': ejection_fraction,
        'high_blood_pressure': 1 if high_blood_pressure else 0,
        'platelets': platelets,
        'serum_creatinine': creatinine,
        'serum_sodium': sodium,
        'sex': 1 if gender == "Homme" else 0,
        'smoking': 1 if smoking else 0,
        'time': time
    }

    # Conversion en tableau NumPy et mise à l'échelle
    input_array = np.array([list(input_data.values())])
    scaled_input = scaler.transform(input_array)

    # Prédiction
    risk_probability = model.predict_proba(scaled_input)[0][1]
    risk_percentage = risk_probability * 100

    # Affichage des résultats
    st.header("Résultats")
    st.markdown(f"### Risque de mortalité pour {name if name else 'ce patient'}")

    # Jauge de risque
    col1, col2, col3 = st.columns([1, 2, 1])
    with col2:
        fig, ax = plt.subplots(figsize=(8, 2))
        ax.barh(0, 100, color='lightgray', height=0.5)
        ax.barh(0, risk_percentage, color='red' if risk_percentage > 50 else 'orange' if risk_percentage > 30 else 'green', height=0.5)
        ax.set_xlim(0, 100)
        ax.set_yticks([])
        ax.set_xticks([0, 25, 50, 75, 100])
        ax.set_xlabel('Risque (%)')
        st.pyplot(fig)

    # Interprétation
    risk_level = "ÉLEVÉ" if risk_percentage > 50 else "MODÉRÉ" if risk_percentage > 30 else "FAIBLE"
    risk_color = "red" if risk_percentage > 50 else "orange" if risk_percentage > 30 else "green"

    st.markdown(f"### Niveau de risque: <span style='color:{risk_color};font-weight:bold'>{risk_level} ({risk_percentage:.1f}%)</span>", unsafe_allow_html=True)

    # Analyse SHAP
    st.subheader("Facteurs de risque importants")
    shap_values = explainer.shap_values(scaled_input)

    plt.figure(figsize=(10, 8))
    shap.summary_plot(shap_values[1], scaled_input, feature_names=model.feature_names_in_, plot_type="bar", show=False)
    st.pyplot(plt.gcf())

    st.write("Ce graphique montre les facteurs qui contribuent le plus au risque de mortalité pour ce patient.")

    # Recommandations
    st.subheader("Recommandations")
    if risk_percentage > 50:
        st.error("Patient à haut risque - Une hospitalisation ou un suivi immédiat est recommandé.")
    elif risk_percentage > 30:
        st.warning("Patient à risque modéré - Un suivi médical régulier est recommandé.")
    else:
        st.success("Patient à faible risque - Maintenir le suivi médical habituel.")

2025-03-13 10:31:26.406 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-13 10:31:26.416 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-13 10:31:27.308 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-03-13 10:31:27.311 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-13 10:31:27.317 No runtime found, using MemoryCacheStorageManager
2025-03-13 10:31:27.322 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-13 10:31:27.327 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-13 10:31:27.339 Thread 'Ma

In [ ]:
!streamlit run votre_script.py

Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: votre_script.py


In [ ]:
!streamlit run --help

Usage: streamlit run [OPTIONS] TARGET [ARGS]...

  Run a Python script, piping stderr to Streamlit.

  The script can be local or it can be an url. In the latter case, Streamlit
  will download the script to a temporary file and runs this file.

Options:
  --global.disableWidgetStateDuplicationWarning BOOLEAN
                                  By default, Streamlit displays a warning
                                  when a user sets both a widget default value
                                  in the function defining the widget and a
                                  widget value via the widget's key in
                                  `st.session_state`.
                                  
                                  If you'd like to turn off this warning, set
                                  this to True.  [env var: STREAMLIT_GLOBAL_DI
                                  SABLE_WIDGET_STATE_DUPLICATION_WARNING]
  --global.showWarningOnDirectExecution BOOLEAN
                    